In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import datetime
import os
import sys

sys.path.insert(0,"../HUXt/code")
import huxt as H
import huxt_analysis as HA
import huxt_inputs as Hin

### This notebook takes you through the first few steps of using HUXt. A link to the main example notebook is here: https://github.com/University-of-Reading-Space-Science/HUXt/blob/master/code/HUXt_examples.ipynb. 

### Task one: review this notebook, try running parts of it by copy pasting into this new notebook. 

-Identify where you create an input for the boundary condition. 

-Identify where you run the model. 

-Identify where you view the timeseries/radial profile. 

-Identify where you can view the 2D model output.  

-Identify where you can inject a CME. 

### Task two: next, try reproducing example four, and changing the carrington rotation number to se the model change. 

#### For each carrington roation you try, plot the boundary condition on one figure to see how different they are. 
